#### **Python env**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np

from sklearn.linear_model import LinearRegression

#### **Import data**

In [ ]:
df = pd.read_csv("store-sales-time-series-forecasting/book_sales.csv", parse_dates = ["Date"])
df = df.drop(["Paperback"], axis = 1)
df.head()

#### **Linear regression**

The interesting features that could be used to solve this problem are time and lags. In order to solce the problem, maybe a combination of the two could be used.

In [ ]:
df["Time"] = range(0, df.shape[0])

In [ ]:
plt.style.use("seaborn-whitegrid")

plt.rc(
    "figure",
    autolayout = True,
    figsize = (11, 4),
    titlesize = 18,
    titleweight = "bold"
)

plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=16,
    titlepad=10,
)

fig, ax = plt.subplots()
ax.plot("Time", "Hardcover", data = df, color = '0.7')
ax = sns.regplot(x = "Time", y = "Hardcover", data = df, ci = 95, scatter_kws = dict(color = "0.25"))
ax.set_title('Time Plot of Hardcover Sales')
ax.grid(False);

In [ ]:
lr = LinearRegression()

lr.fit(X = df["Time"].values.reshape(-1, 1), y = df["Hardcover"].values)

lr.coef_[0], lr.intercept_

In [ ]:
df["Hardcover"].values

In [ ]:
df["Lag_1"] = df.Hardcover.shift(1)
df.head()

In [ ]:
# dato che si osserva una correlazione tra la variabile e il suo lag, 
# tale lag dovrebbe essere tenuto in considerazione per le analisi.
# Stiamo tenendo in considerazione una dipendenza seriale: il sales di un
# giorno sarà minore di quello successivo.  

fig, ax = plt.subplots()
ax = sns.regplot(x = "Lag_1", y = "Hardcover", data = df[["Lag_1", "Hardcover"]], ci = 95, scatter_kws = dict(color = "0.25"))
ax.set_title('Lag Plot of Hardcover Sales')
ax.grid(False);

In [ ]:
lr = LinearRegression()

lr.fit(X = df["Lag_1"].values[1:].reshape(-1, 1), y = df["Hardcover"].values[1:])

lr.coef_[0], lr.intercept_

In [ ]:
lr = LinearRegression()

lr.fit(X = df[["Time", "Lag_1"]].values[1:,:], y = df["Hardcover"].values[1:])

lr.coef_[0], lr.intercept_

#### **Trend**

A _trend_ represents the change in the mean of a time series and it is the slowest-moving part of a series.

Generally speaking, a trend is a slow-moving and persistent change which could involve the mean but also other measures, like the median. Moreover, 
it could be linear or also a persistent and slow-moving seasonal. 

So, in order to highlight the type of trend, a rolling mean could be performed so that any short-term trend should be deleted. Therefore, the width of
the rolling mean should be larger than the seasonal period.   

In [ ]:
df = pd.read_csv("archive/tunnel.csv")
df["Day"] = pd.to_datetime(df["Day"])
df["Time"] = range(0, df.shape[0])
df.head()

In [ ]:
plt.style.use("seaborn-whitegrid")

plt.rc(
    "figure",
    autolayout = True,
    figsize = (11, 4),
    titlesize = 18,
    titleweight = "bold"
)

plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=16,
    titlepad=10,
)


In [ ]:
fig, ax = plt.subplots()
ax.plot("Time", "NumVehicles", data = df, color = '0.7', zorder = 0)
ax = sns.regplot(x = "Time", y = "NumVehicles", data = df, ci = 95, scatter_kws = dict(color = "0.25"))
ax.set_title('Time Plot of Hardcover Sales')
ax.grid(False);

In [ ]:
moving_average = df.rolling(
    window = 365,
    center = True,
    min_periods = 183
).mean()

ax = df["NumVehicles"].plot()
moving_average["NumVehicles"].plot(ax = ax, linewidth = 3)
ax.grid(False);

# possiamo vedere che il trend è lineare.

In [ ]:
# una volta che il trend è stato identificato, possiamo modellarlo: in questo caso, possiamo ottenere le features
# da un processo deterministico.

from statsmodels.tsa.deterministic import DeterministicProcess

dp = DeterministicProcess(
    index = df.Time,    # regressor
    constant = False,    # bias
    order = 1,          # order of the regression
    drop = True         # avoid collinearity
)

# feature estratte per i dati di training. 

X = dp.in_sample()
X.head(5)

In [ ]:
from sklearn.linear_model import LinearRegression

y = df["NumVehicles"]

model = LinearRegression(fit_intercept = True)
model.fit(X, y)

y_pred = pd.Series(model.predict(X), index = X.index)

X = dp.out_of_sample(steps=30)

y_fore = pd.Series(model.predict(X), index=X.index)

In [ ]:
plt.figure(figsize = (10, 5))
plt.plot(y, color = "red", alpha = .7, label = "Data")
plt.plot(y_pred, color = "black", alpha = .7, label = "In-sample predictions")
plt.plot(y_fore, color = "black", ls = "--", alpha = .7, label = "Out-sample predictions")
plt.legend()
plt.grid(False)

#### **Seasonality**

Oscillazioni attorno alla media in modo regolari e periodiche danno vita alla _stagionalità_. Tendenzialmente, sono legate al calendario naturale.

In base alla frequenze, possiamo definire due modi per definire la stagionalità: se è settimanale o giornaliera, si possono usare le variabili dummy. altrimenti la trasformata i Fourier. Questo perché l'approccio con le dummy si basa sul fatto di considerare tanti regressori quanto il periodo e se quest'ultimo dovesse essere grande, allora i regressori in gioco potrebbero essere eccessivi.

##### **Seasonal plot**

Vengono utilizzati per plottare i dati nello stesso intervallo di tempo.

In [ ]:
def seasonal_plot(X, y, period, freq, ax=None):
    if ax is None:
        _, ax = plt.subplots()
    palette = sns.color_palette("husl", n_colors=X[period].nunique(),)
    ax = sns.lineplot(
        x=freq,
        y=y,
        hue=period,
        data=X,
        ci=False,
        ax=ax,
        palette=palette,
        legend=False,
    )
    ax.set_title(f"Seasonal Plot ({period}/{freq})")
    for line, name in zip(ax.lines, X[period].unique()):
        y_ = line.get_ydata()[-1]
        ax.annotate(
            name,
            xy=(1, y_),
            xytext=(6, 0),
            color=line.get_color(),
            xycoords=ax.get_yaxis_transform(),
            textcoords="offset points",
            size=14,
            va="center",
        )
    return ax

In [ ]:
tunnel = pd.read_csv("archive/tunnel.csv", parse_dates=["Day"])
tunnel.set_index(["Day"], inplace = True)
tunnel.head()

In [ ]:
# un grafico per analizzare la stagionalità è il seasonal plot.

X = tunnel.copy()

# day in a week.
X["day"] = X.index.day_of_week
X["week"] = X.index.week
# day in a year.
X["dayofyear"] = X.index.dayofyear
X["year"] = X.index.year

fig, (ax0, ax1) = plt.subplots(2, 1, figsize=(11, 6))
seasonal_plot(X, y="NumVehicles", period="week", freq="day", ax=ax0)
seasonal_plot(X, y="NumVehicles", period="year", freq="dayofyear", ax=ax1);

##### **Dummy**

Per tenere in considerazione le stagionalità corte nella regressione, è possibile tenere 
in considerazione il fatto che per alcuni momenti della stagionalità, come le ore o il giorno,
il valore medio della serie puù aumentare o diminuire. Per tenere in considerazione questo aspetto,
è possiile sfruttare le dummy. Si ricordi però che nel one-hot-encoding una variabile dummy deve essere
eliminata se si considera l'intercetta. 

##### **Periodigramma**

Se il periodo è piuttosto lungo, dovrei considerare  molti regressori. Di conseguenza, si utilizza la trasformata di Fourier in modo da considerare la overall shape della stagionalità. 

Per capire quante coppie sin/cos considerare, si utilizza il periodigramma il quale mostra $(a**2+b**2) / 2$ dove $a$ e $b$ rappresentano i coeff di Fourier per una frequenza fissata. Ovviamente questo è un risultato globale, poi è l'utente a dire in che modo dovranno essere modellate tali frequenze. 

In [ ]:

def plot_periodogram(ts, detrend='linear', ax=None):
    from scipy.signal import periodogram
    fs = pd.Timedelta("1Y") / pd.Timedelta("1D")
    freqencies, spectrum = periodogram(
        ts,
        fs=fs,
        detrend=detrend,
        window="boxcar",
        scaling='spectrum',
    )
    if ax is None:
        _, ax = plt.subplots()
    ax.step(freqencies, spectrum, color="purple")
    ax.set_xscale("log")
    ax.set_xticks([1, 2, 4, 6, 12, 26, 52, 104])
    ax.set_xticklabels(
        [
            "Annual (1)",
            "Semiannual (2)",
            "Quarterly (4)",
            "Bimonthly (6)",
            "Monthly (12)",
            "Biweekly (26)",
            "Weekly (52)",
            "Semiweekly (104)",
        ],
        rotation=30,
    )
    ax.ticklabel_format(axis="y", style="sci", scilimits=(0, 0))
    ax.set_ylabel("Variance")
    ax.set_title("Periodogram")
    return ax

plot_periodogram(tunnel.NumVehicles);

# si osserva che vi è una forte stagionalità settimanale ma non annuale.

In [ ]:
tunnel.index.to_period("D")

In [ ]:
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess


fourier = CalendarFourier(freq = "A", order = 10)

dp = DeterministicProcess(
    index=tunnel.index.to_period("D"),
    constant=True,               # dummy feature for bias (y-intercept)
    order=1,                     # trend (order 1 means linear)
    seasonal=True,               # weekly seasonality (indicators)
    additional_terms=[fourier],  # annual seasonality (fourier)
    drop=True,                   # drop terms to avoid collinearity
)

X = dp.in_sample()
X_fore = dp.out_of_sample(steps=90)

In [ ]:
y = tunnel["NumVehicles"]

model = LinearRegression(fit_intercept=False)
_ = model.fit(X, y)

y_pred = pd.Series(model.predict(X), index=y.index)

y_fore = pd.Series(model.predict(X_fore), index=X_fore.index)

In [ ]:
ax = y.plot(color='0.25', style='.', title="Tunnel Traffic - Seasonal Forecast")
ax = y_pred.plot(ax=ax, label="Seasonal")
ax = y_fore.plot(ax=ax, label="Seasonal Forecast", color='C3')
_ = ax.legend()